<a href="https://colab.research.google.com/github/tansa1024/Open_Project_Winter_2025/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Assignment: Quantum Measurement Dataset Foundations

Build a reproducible tomography workflow that scales from single qubit calibration studies to multi qubit benchmarks. Begin by setting up your environment locally (with OS-specific guidance) or in Google Colab, then generate measurement outcomes using Symmetric Informationally Complete POVMs (SIC POVMs) or Pauli projective measurements. Extend the pipeline with random circuits and document the trade offs you observe.

**Task roadmap**
1. Set up and document your environment.
2. Review the Born rule plus SIC POVM and Pauli projective measurement theory.
3. Generate and visualize QST datasets.
4. Perform single qubit tomography
5. Validate reconstructions, summarize findings, and package deliverables.

> Collaboration on planning is allowed, but every artifact you submit must be authored and executed by you.

## Task 1 · Environment Setup
**Choose one deployment path and capture the exact commands you run.**

### Local virtual environment (recommended)
- **macOS / Linux:**
  1. `python3 -m venv .venv`
  2. `source .venv/bin/activate`
  3. `python -m pip install --upgrade pip wheel`
- **Windows (PowerShell):**
  1. `py -3 -m venv .venv`
  2. `.venv\Scripts\Activate.ps1`
  3. `python -m pip install --upgrade pip wheel`

### Google Colab fallback
- Create a new notebook at https://colab.research.google.com and enable a GPU if available.
- Install the required libraries in the first cell (see the pip example below).
- Save the executed notebook to Drive and export a copy for submission evidence.

### Required baseline packages
- qiskit/pennylane (or an equivalent simulator such as cirq or qutip)
- numpy, scipy, pandas
- plotly (interactive visualization)
- tqdm (progress bars) plus any other support tooling you need


In [1]:
# Run inside your activated virtual environment or a Colab cell.
# Feel free to adjust versions based on your simulator choice.
!python -m pip install qiskit numpy scipy pandas plotly tqdm nbformat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.6 MB/s eta 0:00:00


In [2]:
!python -m pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 117.6 MB/s eta 0:00:00


## Task 2 · Measurement Theory Primer
### Born rule recap
- For a state described by density matrix ρ and measurement operator M_k, the probability of outcome k is `p(k) = Tr(M_k ρ)`.
- For projective measurements, `M_k = P_k` with `P_k^2 = P_k` and `∑_k P_k = I`. For POVMs, `M_k = E_k` where each `E_k` is positive semi-definite and `∑_k E_k = I`.
- Document a short derivation or reference plus a numerical completeness check for your operators.

### SIC POVM vs. Pauli projective (single qubit)
- **SIC POVM strengths:** informational completeness with only four outcomes, symmetric structure, resilience to certain noise.
- **SIC POVM trade-offs:** hardware calibration overhead, non-standard measurement bases, denser classical post-processing.
- **Pauli projective strengths:** hardware-native eigenbases, easier interpretation, wide toolkit support.
- **Pauli projective trade-offs:** requires multiple bases (X/Y/Z) for completeness, higher shot budgets, basis-alignment sensitivity.

Use the `build_measurement_model` stub to serialize your chosen operators (matrices, normalization logs, metadata). Summarize the pros/cons in your notes and justify the model (or hybrid) you adopt for tomography.

### Born Rule
The Born rule states that the probability of measuring a specific outcome in a quantum system is given by the squared magnitude of the probability amplitude associated with that outcome. Mathematically, for a state $\rho$ and a measurement operator $M_k$, the probability of outcome $k$ is $p(k) = Tr(M_k \rho)$.

### Types of Measurement Operators
1.  **Projective Measurements:** These are represented by projection operators $P_k$, where $P_k^2 = P_k$ and $\sum_k P_k = I$ (identity operator). Each $P_k$ corresponds to a distinct measurement outcome.
2.  **POVMs (Positive-Operator Valued Measures):** These are represented by a set of positive semi-definite operators $E_k$, where $\sum_k E_k = I$. POVMs are a more general form of quantum measurement than projective measurements.

### Why Pauli Projective Measurements?
I am choosing Pauli projective measurements due to their **hardware-native eigenbases**, which makes them generally easier to implement and interpret on current quantum hardware. They also have **wide toolkit support**, simplifying the coding and analysis aspects of the tomography. While they require multiple measurement bases (X, Y, Z) for informational completeness, this is often a practical trade-off for their ease of use and direct physical interpretation.

### Reference single-qubit states
Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., `( |0⟩ + i |1⟩ ) / √2`). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

In [3]:
from qiskit import QuantumCircuit
from qiskit.qasm3 import dumps
import json

def create_state_circuit(state_name: str) -> QuantumCircuit:
    qc = QuantumCircuit(1, name=state_name)
    if state_name == '|0⟩':
        pass
    elif state_name == '|1⟩':
        qc.x(0)
    elif state_name == '|+⟩':
        qc.h(0)
    elif state_name == '|-⟩':
        qc.x(0)
        qc.h(0)
    elif state_name == '( |0⟩ + i |1⟩ ) / √2':
        qc.h(0)
        qc.s(0)
    else:
        raise ValueError(f"Unknown state: {state_name}")
    return qc
state_names = [
    '|0⟩',
    '|1⟩',
    '|+⟩',
    '|-⟩',
    '( |0⟩ + i |1⟩ ) / √2'
]

prepared_states_info = {}
for name in state_names:
    circuit = create_state_circuit(name)
    gate_summary = {
        "gates": [
            {
                "name": inst.operation.name,
                "qubits": [circuit.find_bit(q).index for q in inst.qubits]
            }
            for inst in circuit.data
        ],
        "openqasm3": dumps(circuit)
    }

    prepared_states_info[name] = {
        "gate_summary": gate_summary
    }
    print("\n==============================")
    print(f"State: {name}")
    print("==============================")
    print("Circuit Diagram:")
    print(circuit.draw(output='text'))
    print("Gate Summary:")
    print(json.dumps(gate_summary["gates"], indent=2))

with open("reference_single_qubit_states.json", "w") as f:
    json.dump(prepared_states_info, f, indent=4)

print("\n Reference state library saved successfully.")


State: |0⟩
Circuit Diagram:
   
q: 
   
Gate Summary:
[]

State: |1⟩
Circuit Diagram:
   ┌───┐
q: ┤ X ├
   └───┘
Gate Summary:
[
  {
    "name": "x",
    "qubits": [
      0
    ]
  }
]

State: |+⟩
Circuit Diagram:
   ┌───┐
q: ┤ H ├
   └───┘
Gate Summary:
[
  {
    "name": "h",
    "qubits": [
      0
    ]
  }
]

State: |-⟩
Circuit Diagram:
   ┌───┐┌───┐
q: ┤ X ├┤ H ├
   └───┘└───┘
Gate Summary:
[
  {
    "name": "x",
    "qubits": [
      0
    ]
  },
  {
    "name": "h",
    "qubits": [
      0
    ]
  }
]

State: ( |0⟩ + i |1⟩ ) / √2
Circuit Diagram:
   ┌───┐┌───┐
q: ┤ H ├┤ S ├
   └───┘└───┘
Gate Summary:
[
  {
    "name": "h",
    "qubits": [
      0
    ]
  },
  {
    "name": "s",
    "qubits": [
      0
    ]
  }
]

 Reference state library saved successfully.


In [4]:
from typing import Dict, Any
import pathlib
import numpy as np

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    pauli_x = np.array([[0, 1], [1, 0]], dtype=complex)
    pauli_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    pauli_z = np.array([[1, 0], [0, -1]], dtype=complex)
    identity = np.array([[1, 0], [0, 1]], dtype=complex)
    px0 = 0.5 * (identity + pauli_x)
    px1 = 0.5 * (identity - pauli_x)
    pz0 = 0.5 * (identity + pauli_z)
    pz1 = 0.5 * (identity - pauli_z)
    py0 = 0.5 * (identity + pauli_y)
    py1 = 0.5 * (identity - pauli_y)

    measurement_operators = {
        'Z_basis': [pz0, pz1],
        'X_basis': [px0, px1],
        'Y_basis': [py0, py1]
    }
    #Checking normalization: sum of operators should equal identity
    normalization_checks = {}
    for basis, ops in measurement_operators.items():
        sum_ops = np.sum(ops, axis=0)
        normalization_checks[basis] = {
            'sum_to_identity': np.allclose(sum_ops, identity),
            'sum_matrix': sum_ops.tolist()
        }

    metadata = {
        'measurement_model_type': 'Pauli Projective',
        'qubits': 1,
        'description': 'Single-qubit Pauli X, Y, Z projective measurements.'
    }

    model = {
        'operators': {
            'Z0': pz0.tolist(),
            'Z1': pz1.tolist(),
            'X0': px0.tolist(),
            'X1': px1.tolist(),
            'Y0': py0.tolist(),
            'Y1': py1.tolist(),
        },
        'normalization_checks': normalization_checks,
        'metadata': metadata
    }
    return model
pauli_measurement_model = build_measurement_model(config_path=None)
print("Proved that pauli operators sum to I")
print("Pauli Measurement Model stored in 'pauli_measurement_model' variable.")

Proved that pauli operators sum to I
Pauli Measurement Model stored in 'pauli_measurement_model' variable.


In [5]:
#@title helper functions for density matrix visualization

import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),  # bottom
    (4, 5, 6), (4, 6, 7),  # top
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
 )

def _phase_to_pi_string(angle_rad: float) -> str:
    """Format a phase angle as a simplified multiple of π."""
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    numerator = frac.numerator
    denominator = frac.denominator
    sign = "-" if numerator < 0 else ""
    numerator = abs(numerator)
    if denominator == 1:
        magnitude = f"{numerator}" if numerator != 1 else ""
    else:
        magnitude = f"{numerator}/{denominator}"
    return f"{sign}{magnitude}π" if magnitude else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix (|ρ_ij| as bar height, phase as color)"):
    """Render a density matrix as a grid of solid histogram bars with phase coloring."""
    rho = np.asarray(rho)
    if rho.ndim != 2 or rho.shape[0] != rho.shape[1]:
        raise ValueError("rho must be a square matrix")

    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)
    x_vals = np.arange(dim)
    y_vals = np.arange(dim)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False
    for i in range(dim):
        for j in range(dim):
            height = mags[i, j]
            phase = phases[i, j]
            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45
            vertices = (
                (x0, y0, 0.0), (x1, y0, 0.0), (x1, y1, 0.0), (x0, y1, 0.0),
                (x0, y0, height), (x1, y0, height), (x1, y1, height), (x0, y1, height)
            )
            x_coords, y_coords, z_coords = zip(*vertices)
            i_idx, j_idx, k_idx = zip(*_CUBE_FACES)
            phase_pi = _phase_to_pi_string(phase)
            mesh = go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i_idx,
                j=j_idx,
                k=k_idx,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase ",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None,
                opacity=1.0,
                flatshading=False,
                hovertemplate=
                    f"i={i}, j={j}<br>|ρ_ij|={height:.3f}<br>arg(ρ_ij)={phase_pi}<extra></extra>",
                lighting=dict(ambient=0.6, diffuse=0.7)
            )
            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title="i",
                tickmode="array",
                tickvals=x_vals,
                ticktext=basis_labels
            ),
            yaxis=dict(
                title="j",
                tickmode="array",
                tickvals=y_vals,
                ticktext=basis_labels
            ),
            zaxis=dict(title="|ρ_ij|"),
            aspectratio=dict(x=1, y=1, z=0.7)
        ),
        title=title,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()


### Visualization helpers
Use the histogram helper below to inspect reconstructed density matrices. Include screenshots or exported HTML for a few representative states in your report.

In [6]:
# Demonstration: random 2-qubit density matrix
dim = 4
A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
rho = A @ A.conj().T
rho = rho / np.trace(rho)  # normalize

labels = ["00", "01", "10", "11"]
plot_density_matrix_histogram(rho, basis_labels=labels, title="Random 2-qubit state (density matrix)")

In [7]:
#@title helper function Demonstration: canonical Bell states
bell_states = {
    "Φ⁺": np.array([1, 0, 0, 1], dtype=complex) / np.sqrt(2),
    "Φ⁻": np.array([1, 0, 0, -1], dtype=complex) / np.sqrt(2),
    "Ψ⁺": np.array([0, 1, 1, 0], dtype=complex) / np.sqrt(2),
    "Ψ⁻": np.array([0, 1, -1, 0], dtype=complex) / np.sqrt(2)
}

for name, state in bell_states.items():
    density_matrix = np.outer(state, state.conj())
    plot_density_matrix_histogram(
        density_matrix,
        basis_labels=["00", "01", "10", "11"],
        title=f"Bell state {name} (density matrix)"
    )

## Task 3 · QST Data generation
- use random circuits or bonus points for using gen Ai to produce realistic quantum circuits
- For each reference state you prepared, execute shots under your chosen measurement model using chosen quantum simulator. Record raw counts and computed probabilities.
- Store measurement data (`single_qubit_<state>.npx` or `.npy`)

In [8]:
from typing import List, Dict
from dataclasses import dataclass
import pathlib

def complex_matrix_to_json(matrix):
    return [
        [
            {"real": float(z.real), "imag": float(z.imag)}
            for z in row
        ]
        for row in matrix
    ]

@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path
def get_reference_states() -> Dict[str, np.ndarray]:
    s0 = np.array([1, 0], dtype=complex)
    s1 = np.array([0, 1], dtype=complex)
    s_plus = (1/np.sqrt(2)) * np.array([1, 1], dtype=complex)
    s_minus = (1/np.sqrt(2)) * np.array([1, -1], dtype=complex)
    s_iplus = (1/np.sqrt(2)) * np.array([1, 1j], dtype=complex)
    return {
        "0": np.outer(s0, s0.conj()),
        "1": np.outer(s1, s1.conj()),
        "plus": np.outer(s_plus, s_plus.conj()),
        "minus": np.outer(s_minus, s_minus.conj()),
        "iplus": np.outer(s_iplus, s_iplus.conj()),
    }
def generate_measurement_dataset(variants: List[DatasetVariant], num_shots: int = 8192) -> None:
    reference_states = get_reference_states()
    measurement_operators = build_measurement_model(pathlib.Path("data/temp_config.json"))
    for variant in variants:
        variant.measurement_data_path.parent.mkdir(parents=True, exist_ok=True)
        variant.density_matrix_path.parent.mkdir(parents=True, exist_ok=True)
        state_key = variant.name
        basis_key = variant.measurement_model.replace("Pauli_", "")
        if state_key not in reference_states or basis_key not in measurement_operators['operators']:
            continue

        rho_target = reference_states[state_key]
        b_prefix = basis_key[0]
        p0 = np.array(measurement_operators['operators'][f"{b_prefix}0"])
        p1 = np.array(measurement_operators['operators'][f"{b_prefix}1"])
        projectors = [p0, p1]
        probs = [np.real(np.trace(p @ rho_target)) for p in projectors]
        probs = np.array(probs) / np.sum(probs)
        counts = np.random.multinomial(num_shots, probs)
        np.save(variant.measurement_data_path, counts)
        np.save(variant.density_matrix_path, rho_target)
        metadata = {
            "variant_name": variant.name,
            "basis": basis_key,
            "shots": num_shots,
            "counts": counts.tolist(),
            "probabilities": probs.tolist(),
            "projectors": [complex_matrix_to_json(p) for p in projectors]
        }

        with open(variant.metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)

    print(f" Generated {len(variants)} data variants in {variant.measurement_data_path.parent}")

In [9]:
all_variants = []
base_dir = pathlib.Path("data/single_qubit")

for s_name in ["0", "1", "plus", "minus", "iplus"]:
    for b_name in ["X_basis", "Y_basis", "Z_basis"]:
        v = DatasetVariant(
            name=s_name,
            circuit_summary=f"Preparation of {s_name}",
            measurement_model=f"Pauli_{b_name}",
            measurement_data_path=base_dir / f"{s_name}_{b_name}_counts.npy",
            metadata_path=base_dir / f"{s_name}_{b_name}_meta.json",
            density_matrix_path=base_dir / f"{s_name}_rho_ideal.npy"
        )
        all_variants.append(v)

generate_measurement_dataset(all_variants)

 Generated 15 data variants in data/single_qubit


In [10]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import json
import os

NUM_SHOTS = 8192
simulator = AerSimulator()
output_dir = "data/single_qubit"
os.makedirs(output_dir, exist_ok=True)

measurement_results = {}

for state_name, info in prepared_states_info.items():
    base_circuit = create_state_circuit(state_name)
    state_results = {"raw_counts": {}, "probabilities": {}, "density_matrix_ground_truth": {}}
    statevector_simulator = AerSimulator(method='statevector')
    sv_circuit = base_circuit.copy()
    sv_circuit.save_statevector()

    job = statevector_simulator.run(sv_circuit, shots=1)
    result = job.result()
    statevector = result.get_statevector(sv_circuit)
    rho_ground_truth = np.outer(statevector, statevector.conjugate())
    state_results["density_matrix_ground_truth"] = complex_matrix_to_json(rho_ground_truth)
    filename_prefix_for_npy = state_name.replace(' ', '_').replace('|', '').replace('⟩', '').replace('(', '').replace(')', '').replace('/', '_').replace('+', 'plus').replace('-', 'minus').replace('√2', 'sqrt2').replace('i', 'i_')
    rho_ground_truth_path = os.path.join(output_dir, f"rho_ground_truth_{filename_prefix_for_npy}.npy")
    np.save(rho_ground_truth_path, rho_ground_truth)
    print(f"Ground truth density matrix for {state_name} saved to {rho_ground_truth_path}")

    print(f"\n--- Simulating measurements for {state_name} ---")
    for basis_name in ['Z_basis', 'X_basis', 'Y_basis']:
        meas_circuit = base_circuit.copy()

        if basis_name == 'X_basis':
            meas_circuit.h(0)
        elif basis_name == 'Y_basis':
            meas_circuit.sdg(0)
            meas_circuit.h(0)

        meas_circuit.measure_all()

        transpiled_circuit = transpile(meas_circuit, simulator)
        job = simulator.run(transpiled_circuit, shots=NUM_SHOTS)
        result = job.result()
        counts = result.get_counts(transpiled_circuit)

        probabilities = {k: v / NUM_SHOTS for k, v in counts.items()}

        state_results["raw_counts"][basis_name] = counts
        state_results["probabilities"][basis_name] = probabilities
        counts_path = os.path.join(output_dir, f"counts_{filename_prefix_for_npy}_{basis_name}.npy")
        counts_array = np.array([counts.get('0', 0), counts.get('1', 0)])
        np.save(counts_path, counts_array)
        print(f"  Raw counts for {state_name} in {basis_name} saved to {counts_path}")

        print(f"  {basis_name} counts: {counts}")
        print(f"  {basis_name} probabilities: {probabilities}")

    measurement_results[state_name] = state_results
    results_path = os.path.join(output_dir, f"single_qubit_{filename_prefix_for_npy}_measurements.json")
    with open(results_path, 'w') as f:
        json.dump(state_results, f, indent=4)
    print(f"Aggregated measurement data for {state_name} saved to {results_path}")

print("\n--- Data generation complete ---")

Ground truth density matrix for |0⟩ saved to data/single_qubit/rho_ground_truth_0.npy

--- Simulating measurements for |0⟩ ---
  Raw counts for |0⟩ in Z_basis saved to data/single_qubit/counts_0_Z_basis.npy
  Z_basis counts: {'0': 8192}
  Z_basis probabilities: {'0': 1.0}
  Raw counts for |0⟩ in X_basis saved to data/single_qubit/counts_0_X_basis.npy
  X_basis counts: {'1': 4146, '0': 4046}
  X_basis probabilities: {'1': 0.506103515625, '0': 0.493896484375}
  Raw counts for |0⟩ in Y_basis saved to data/single_qubit/counts_0_Y_basis.npy
  Y_basis counts: {'0': 4021, '1': 4171}
  Y_basis probabilities: {'0': 0.4908447265625, '1': 0.5091552734375}
Aggregated measurement data for |0⟩ saved to data/single_qubit/single_qubit_0_measurements.json
Ground truth density matrix for |1⟩ saved to data/single_qubit/rho_ground_truth_1.npy

--- Simulating measurements for |1⟩ ---
  Raw counts for |1⟩ in Z_basis saved to data/single_qubit/counts_1_Z_basis.npy
  Z_basis counts: {'1': 8192}
  Z_basis prob

In [11]:

prefix_to_state_name = {}
for name in state_names:
    prefix_part = name.replace(' ', '_').replace('|', '').replace('⟩', '').replace('(', '').replace(')', '').replace('/', '_').replace('+', 'plus').replace('-', 'minus').replace('√2', 'sqrt2').replace('i', 'i_')
    prefix_to_state_name[prefix_part] = name
loaded_measurement_data = {}

print(f"Attempting to load data from: {output_dir}")
try:
    files = os.listdir(output_dir)
except FileNotFoundError:
    print(f"Error: Directory '{output_dir}' not found. Please ensure the previous simulation step was executed.")
    files = []

for filename in files:
    if filename.startswith("single_qubit_") and filename.endswith("_measurements.json"):
        full_path = os.path.join(output_dir, filename)
        extracted_prefix = filename[len("single_qubit_"):-len("_measurements.json")]

        if extracted_prefix in prefix_to_state_name:
            original_state_name = prefix_to_state_name[extracted_prefix]
            with open(full_path, 'r') as f:
                data = json.load(f)
                loaded_measurement_data[original_state_name] = data
                print(f"Loaded data for state: '{original_state_name}' from file: '{filename}'")
        else:
            print(f"Warning: Filename prefix '{extracted_prefix}' from '{filename}' does not match any known state. Skipping.")

print(f"\nSuccessfully loaded {len(loaded_measurement_data)} single-qubit measurement datasets.")


Attempting to load data from: data/single_qubit
Loaded data for state: '|+⟩' from file: 'single_qubit_plus_measurements.json'
Loaded data for state: '( |0⟩ + i |1⟩ ) / √2' from file: 'single_qubit__0_plus_i__1____sqrt2_measurements.json'
Loaded data for state: '|0⟩' from file: 'single_qubit_0_measurements.json'
Loaded data for state: '|1⟩' from file: 'single_qubit_1_measurements.json'
Loaded data for state: '|-⟩' from file: 'single_qubit_mi_nus_measurements.json'

Successfully loaded 5 single-qubit measurement datasets.


## Task 4 · Single-Qubit Tomography
- Synthesize the reference states from Task 2 (|0⟩, |1⟩, |+⟩, |−⟩, phase-offset) plus any noisy variants you want to study.
- For each state, generate measurement shots using your chosen model (SIC POVM, Pauli axes, or a hybrid). Capture raw counts, probabilities, and seeds.
- Reconstruct the density matrix via linear inversion or maximum-likelihood estimation. Compare results across measurement models when possible.
- Quantify reconstruction fidelity (e.g., fidelity, trace distance, Bloch vector error) and tabulate the metrics.
- save data under `data/single_qubit/`: measurement outcomes (`.npx`/`.npy`), reconstructions, metadata (JSON/Markdown), and helper visualizations created with `plot_density_matrix_histogram`.

In [12]:
pauli_measurement_operators = pauli_measurement_model['operators']
PZ0 = np.array(pauli_measurement_operators['Z0'])
PZ1 = np.array(pauli_measurement_operators['Z1'])
PX0 = np.array(pauli_measurement_operators['X0'])
PX1 = np.array(pauli_measurement_operators['X1'])
PY0 = np.array(pauli_measurement_operators['Y0'])
PY1 = np.array(pauli_measurement_operators['Y1'])

pauli_x_matrix = np.array([[0, 1], [1, 0]], dtype=complex)
pauli_y_matrix = np.array([[0, -1j], [1j, 0]], dtype=complex)
pauli_z_matrix = np.array([[1, 0], [0, -1]], dtype=complex)
identity_matrix = np.array([[1, 0], [0, 1]], dtype=complex)

In [13]:
def linear_inversion_tomography(
    counts_data: Dict[str, Dict[str, int]],
    pauli_x: np.ndarray,
    pauli_y: np.ndarray,
    pauli_z: np.ndarray,
    identity_matrix: np.ndarray,
    num_shots: int
) -> np.ndarray:
    p_Z0 = counts_data['Z_basis'].get('0', 0) / num_shots
    p_Z1 = counts_data['Z_basis'].get('1', 0) / num_shots

    p_X0 = counts_data['X_basis'].get('0', 0) / num_shots
    p_X1 = counts_data['X_basis'].get('1', 0) / num_shots

    p_Y0 = counts_data['Y_basis'].get('0', 0) / num_shots
    p_Y1 = counts_data['Y_basis'].get('1', 0) / num_shots

    rx = p_X0 - p_X1
    ry = p_Y0 - p_Y1
    rz = p_Z0 - p_Z1

    rho_reconstructed = 0.5 * (identity_matrix + rx * pauli_x + ry * pauli_y + rz * pauli_z)
    return rho_reconstructed

print("linear_inversion_tomography function defined successfully.")

linear_inversion_tomography function defined successfully.


In [14]:
reconstructed_density_matrices = {}
for state_name, data in loaded_measurement_data.items():
    counts_data = data['raw_counts']
    rho_reconstructed = linear_inversion_tomography(
        counts_data=counts_data,
        pauli_x=pauli_x_matrix,
        pauli_y=pauli_y_matrix,
        pauli_z=pauli_z_matrix,
        identity_matrix=identity_matrix,
        num_shots=NUM_SHOTS
    )

    if 'density_matrix_ground_truth' not in data:
        print(f"Error: 'density_matrix_ground_truth' not found for state {state_name}. Available keys: {data.keys()}")
        continue

    rho_ground_truth_json = data['density_matrix_ground_truth']
    rho_ground_truth = np.array([[complex(val['real'], val['imag']) for val in row] for row in rho_ground_truth_json])

    reconstructed_density_matrices[state_name] = {
        "reconstructed_rho": rho_reconstructed,
        "ground_truth_rho": rho_ground_truth
    }

    print(f"\n--- Reconstructed Density Matrix for {state_name} ---")
    print("Reconstructed:")
    print(np.round(rho_reconstructed, 4))
    print("Ground Truth:")
    print(np.round(rho_ground_truth, 4))

print("\nDensity matrix reconstruction completed for all states. Results stored in 'reconstructed_density_matrices'.")


--- Reconstructed Density Matrix for |+⟩ ---
Reconstructed:
[[0.4949+0.j     0.5   +0.0065j]
 [0.5   -0.0065j 0.5051+0.j    ]]
Ground Truth:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]

--- Reconstructed Density Matrix for ( |0⟩ + i |1⟩ ) / √2 ---
Reconstructed:
[[0.4979+0.j  0.0038-0.5j]
 [0.0038+0.5j 0.5021+0.j ]]
Ground Truth:
[[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]]

--- Reconstructed Density Matrix for |0⟩ ---
Reconstructed:
[[ 1.    +0.j     -0.0061+0.0092j]
 [-0.0061-0.0092j  0.    +0.j    ]]
Ground Truth:
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]

--- Reconstructed Density Matrix for |1⟩ ---
Reconstructed:
[[0.    +0.j     0.0055-0.0034j]
 [0.0055+0.0034j 1.    +0.j    ]]
Ground Truth:
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

--- Reconstructed Density Matrix for |-⟩ ---
Reconstructed:
[[ 0.4919+0.j     -0.5   -0.0042j]
 [-0.5   +0.0042j  0.5081+0.j    ]]
Ground Truth:
[[ 0.5-0.j -0.5-0.j]
 [-0.5+0.j  0.5-0.j]]

Density matrix reconstruction completed for all states. Results stored in 'recons

In [15]:
import os

output_dir = "data/single_qubit/visualizations"
os.makedirs(output_dir, exist_ok=True)

basis_labels_single_qubit = ['0', '1']
for state_name, data in reconstructed_density_matrices.items():
    rho_reconstructed = data['reconstructed_rho']
    rho_ground_truth = data['ground_truth_rho']
    fig_reconstructed = go.Figure(plot_density_matrix_histogram(
        rho_reconstructed,
        basis_labels=basis_labels_single_qubit,
        title=f"Reconstructed Density Matrix for {state_name}"
    ))
    file_safe_name = state_name.replace("|", "").replace("⟩", "").replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace("+", "plus").replace("-", "minus").replace("√2", "sqrt2").replace("i", "i_")
    reconstructed_plot_path = os.path.join(output_dir, f"reconstructed_rho_{file_safe_name}.html")
    fig_reconstructed.write_html(reconstructed_plot_path)
    print(f"Saved reconstructed density matrix plot for {state_name} to {reconstructed_plot_path}")

    fig_ground_truth = go.Figure(plot_density_matrix_histogram(
        rho_ground_truth,
        basis_labels=basis_labels_single_qubit,
        title=f"Ground Truth Density Matrix for {state_name}"
    ))
    ground_truth_plot_path = os.path.join(output_dir, f"ground_truth_rho_{file_safe_name}.html")
    fig_ground_truth.write_html(ground_truth_plot_path)
    print(f"Saved ground truth density matrix plot for {state_name} to {ground_truth_plot_path}")

print("Visualizations completed and saved for all states.")

Saved reconstructed density matrix plot for |+⟩ to data/single_qubit/visualizations/reconstructed_rho_plus.html


Saved ground truth density matrix plot for |+⟩ to data/single_qubit/visualizations/ground_truth_rho_plus.html


Saved reconstructed density matrix plot for ( |0⟩ + i |1⟩ ) / √2 to data/single_qubit/visualizations/reconstructed_rho__0_plus_i__1____sqrt2.html


Saved ground truth density matrix plot for ( |0⟩ + i |1⟩ ) / √2 to data/single_qubit/visualizations/ground_truth_rho__0_plus_i__1____sqrt2.html


Saved reconstructed density matrix plot for |0⟩ to data/single_qubit/visualizations/reconstructed_rho_0.html


Saved ground truth density matrix plot for |0⟩ to data/single_qubit/visualizations/ground_truth_rho_0.html


Saved reconstructed density matrix plot for |1⟩ to data/single_qubit/visualizations/reconstructed_rho_1.html


Saved ground truth density matrix plot for |1⟩ to data/single_qubit/visualizations/ground_truth_rho_1.html


Saved reconstructed density matrix plot for |-⟩ to data/single_qubit/visualizations/reconstructed_rho_mi_nus.html


Saved ground truth density matrix plot for |-⟩ to data/single_qubit/visualizations/ground_truth_rho_mi_nus.html
Visualizations completed and saved for all states.


## Task 5 · Validation and Reporting
- Compare reconstructed density matrices against the actual density matrices using fidelity, trace distance, or other suitable metrics. Plot trends (per circuit depth, shot count, or measurement model).
- Highlight sources of error (shot noise, model mismatch, simulator approximations) and describe mitigation strategies you tested or plan to try.
- Summarize outcomes in a short technical report or table
- Include at least one qualitative visualization (e.g., density-matrix histograms or Bloch-sphere plots) for both single- and multi-qubit cases.
- Close with a brief reflection covering tooling friction, open questions, and ideas for Week 2 in markdown cell.

In [16]:
import numpy as np
from scipy.linalg import sqrtm
from scipy.linalg import eigvalsh

def calculate_trace_distance(rho: np.ndarray, sigma: np.ndarray) -> float:
    diff = rho - sigma
    eigenvalues = eigvalsh(diff)
    trace_distance_val = 0.5 * np.sum(np.abs(eigenvalues))
    return np.real(trace_distance_val)
print("Trace distance calculation function defined.")

def calculate_fidelity(rho: np.ndarray, sigma: np.ndarray) -> float:
    sqrt_rho = sqrtm(rho)
    intermediate_matrix = sqrt_rho @ sigma @ sqrt_rho
    fidelity_val = np.trace(sqrtm(intermediate_matrix))
    return np.real(fidelity_val * fidelity_val)
metrics_summary = {}
for state_name, data in reconstructed_density_matrices.items():
    rho_reconstructed = data['reconstructed_rho']
    rho_ground_truth = data['ground_truth_rho']
    fidelity = calculate_fidelity(rho_reconstructed, rho_ground_truth)
    trace_distance = calculate_trace_distance(rho_reconstructed, rho_ground_truth)
    reconstructed_density_matrices[state_name]['fidelity'] = fidelity
    reconstructed_density_matrices[state_name]['trace_distance'] = trace_distance
    metrics_summary[state_name] = {
        "fidelity": fidelity,
        "trace_distance": trace_distance
    }

    print(f"\n--- Metrics for {state_name} ---")
    print(f"  Fidelity: {fidelity:.4f}")
    print(f"  Trace Distance: {trace_distance:.4f}")

print("\nFidelity and trace distance calculations completed and stored.")

Trace distance calculation function defined.

--- Metrics for |+⟩ ---
  Fidelity: 1.0000
  Trace Distance: 0.0083

--- Metrics for ( |0⟩ + i |1⟩ ) / √2 ---
  Fidelity: 1.0000
  Trace Distance: 0.0043

--- Metrics for |0⟩ ---
  Fidelity: 1.0000
  Trace Distance: 0.0110

--- Metrics for |1⟩ ---
  Fidelity: 1.0000
  Trace Distance: 0.0065

--- Metrics for |-⟩ ---
  Fidelity: 1.0000
  Trace Distance: 0.0091

Fidelity and trace distance calculations completed and stored.


In [17]:
import json
import os
from pathlib import Path
output_dir = Path('./data/single_qubit')
output_dir.mkdir(parents=True, exist_ok=True)
for state_name, data in reconstructed_density_matrices.items():
    rho_reconstructed = data['reconstructed_rho']
    rho_ground_truth = data['ground_truth_rho']

    fidelity = calculate_fidelity(rho_reconstructed, rho_ground_truth)
    trace_distance = calculate_trace_distance(rho_reconstructed, rho_ground_truth)

    metrics_summary[state_name] = {
        "fidelity": fidelity,
        "trace_distance": trace_distance
    }
    file_safe_name = state_name.replace("|", "").replace("⟩", "").replace(" ", "_").replace("/", "_")
    metadata_filename = f"reconstruction_metadata_{file_safe_name}.json"
    metadata_path = output_dir / metadata_filename

    payload = {
        "state_name": state_name,
        "fidelity": float(fidelity),
        "trace_distance": float(trace_distance),
        "num_shots": 8192
    }

    with open(metadata_path, 'w') as f:
        json.dump(payload, f, indent=4)

    print(f"Saved metrics for {state_name} to {metadata_filename}")
print("\nAll metadata files generated.")

Saved metrics for |+⟩ to reconstruction_metadata_+.json
Saved metrics for ( |0⟩ + i |1⟩ ) / √2 to reconstruction_metadata_(_0_+_i_1_)___√2.json
Saved metrics for |0⟩ to reconstruction_metadata_0.json
Saved metrics for |1⟩ to reconstruction_metadata_1.json
Saved metrics for |-⟩ to reconstruction_metadata_-.json

All metadata files generated.


In [18]:
import json
import pandas as pd
from pathlib import Path
from typing import Sequence

def summarize_validation_runs(result_paths: Sequence[Path]) -> None:
    summary_data = []
    for path in result_paths:
        if path.suffix == '.json':
            try:
                with open(path, 'r') as f:
                    data = json.load(f)
                summary_data.append({
                    "State": data.get("state_name", "Unknown"),
                    "Fidelity": data.get("fidelity", 0.0),
                    "Trace Distance": data.get("trace_distance", 0.0),
                    "Shots": data.get("num_shots", "N/A")
                })
            except Exception as e:
                print(f" Could not process {path.name}: {e}")

    if not summary_data:
        print(" No valid metadata files found. Check your file paths!")
        return
    df = pd.DataFrame(summary_data)
    df = df.sort_values(by="State")
    print("\n" + "="*50)
    print("      QUANTUM STATE TOMOGRAPHY SUMMARY REPORT")
    print("="*50)
    print(df.to_string(index=False))
    print("="*50)
    avg_fidelity = df["Fidelity"].mean()
    print(f"Average Fidelity: {avg_fidelity:.5f}")
    print(f"Average Trace Distance: {df['Trace Distance'].mean():.5f}")
    summary_csv = Path("./data/validation_summary.csv")
    df.to_csv(summary_csv, index=False)
    print(f"\n Summary report saved to: {summary_csv}")

output_dir = Path('./data/single_qubit')
metadata_files = list(output_dir.glob('reconstruction_metadata_*.json'))

summarize_validation_runs(metadata_files)


      QUANTUM STATE TOMOGRAPHY SUMMARY REPORT
               State  Fidelity  Trace Distance  Shots
( |0⟩ + i |1⟩ ) / √2       1.0        0.004316   8192
                 |+⟩       1.0        0.008255   8192
                 |-⟩       1.0        0.009063   8192
                 |0⟩       1.0        0.011003   8192
                 |1⟩       1.0        0.006470   8192
Average Fidelity: 1.00000
Average Trace Distance: 0.00782

 Summary report saved to: data/validation_summary.csv


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Submission Checklist
- Environment setup: env directory (requirements.txt or environment.yml), OS diagnostics, and import verification logs/notebook cells.
- Measurement theory notes: Born rule recap, SIC POVM vs. Pauli analysis, operator definitions, and validation checks.
- Data artifacts: `.npx`/`.npy` files for single- and multi-qubit datasets, metadata summaries, density matrices, and visualization exports.
- Source assets: notebooks/scripts for tomography, dataset generation, validation, and any AI prompt transcripts if used.
- Technical write-up (Markdown ) plus a brief reflection on tools used , open questions, and planned improvements.

-----